## Clean CSV code like a basic clean

In [20]:
import pandas as pd

# Load the CSV into a DataFrame
file_path = 'dbird_data_Georgia_Tech_final(1).csv' 
dfoG = pd.read_csv(file_path)

# Check for duplicate rows
duplicate_rows = dfoG[dfoG.duplicated()]
print(f"Number of duplicate rows: {len(duplicate_rows)}")

# Drop the columns 'notes' and 'image'
dfoG = dfoG.drop(columns=['notes', 'image', 'created_at_utc','updated_at_utc', 'location_accuracy_meters', 'mobile_device_gps_source'])

# Save the updated DataFrame back to a CSV if needed
dfoG.to_csv('updated_dbird.csv', index=False)

#so I know when clena is done
print("Sucessfully cleaned")

Number of duplicate rows: 0
Sucessfully cleaned


Use the new cleaned CSV data to calculate everything

In [21]:
import pandas as pd

file_path = 'updated_dbird.csv'
df = pd.read_csv('updated_dbird.csv')

# Define latitude and longitude bounds
top_latitude = 33.77592
bottom_latitude = 33.77495
left_longitude = -84.40163
right_longitude = -84.40106



In [22]:
# Convert 'observed_at_utc' column to datetime
df['observed_at_utc'] = pd.to_datetime(df['observed_at_utc'], errors='coerce')

# Verify the conversion
df.info()  # Check for datetime type in the 'observed_at_utc' column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   id                  503 non-null    object             
 1   observed_at_utc     271 non-null    datetime64[ns, UTC]
 2   latitude            503 non-null    float64            
 3   longitude           503 non-null    float64            
 4   common_name         503 non-null    object             
 5   scientific_name     503 non-null    object             
 6   ebird_species_code  419 non-null    object             
 7   age                 503 non-null    object             
 8   sex                 503 non-null    object             
 9   bird_status         503 non-null    object             
 10  cause               499 non-null    object             
dtypes: datetime64[ns, UTC](1), float64(2), object(8)
memory usage: 43.4+ KB


In [23]:
# Filter rows within the latitude and longitude range
df_2024_within_bounds = df[
    (df['latitude'] <= top_latitude) &
    (df['latitude'] >= bottom_latitude) &
    (df['longitude'] >= left_longitude) &
    (df['longitude'] <= right_longitude)
]
# Rows in 2024 but outside bounds
df_2024_outside_bounds = df.drop(df_2024_within_bounds.index)

In [24]:
df_2024_within_bounds.to_csv('dbird_2024_within_bounds.csv', index=False)
df_2024_outside_bounds.to_csv('dbird_2024_outside_bounds.csv', index=False)

print("Files saved successfully!")

Files saved successfully!
